In [83]:
import bs4
from urllib.request import urlopen as uReq # to grab the page
from bs4 import BeautifulSoup as soup # to parse html text

In [84]:
# scrape graphics cards

In [146]:
# create a file
filename = "hard_disk.csv"
f = open(filename, "w", encoding='utf-8')

# create a header for the file

headers = "S.no; Stock Id;Image; Brand; Rating; Number of ratings; Description; features; Price; Offers; Shipping Price;\n"
f.write(headers)

# create a url variable
my_url = 'https://www.newegg.com/p/pl?d=graphics+card'

# count the pages dynamically or in this case run over the start to end provided on the page
# run for all the pages
count = 0
for i in range(1,91):
    page_number = str(i)
    
    # create a dynamic url to get the details from each page
    
    uClient = uReq(my_url + '&page=' + page_number)
    page_html = uClient.read()
    uClient.close()

    #html parser
    page_soup = soup(page_html, "html.parser")
    
    # extract the needed data
    containers = page_soup.findAll("div", {"class":"item-cell"})
    len(containers)
    
    for container in containers:
        
        # try catch to prevent the divs which dont have an item - container - to prevent from breaking
        try:
            c = container.find("div", {"class": "item-container"})
            e_image_url = c.a.img["src"]

            item_info = container.find("div", {"class": "item-info"})
            e_item_brand = item_info.find("a", {"class": "item-brand"}).img["title"]

            e_item_rating = item_info.find("a", {"class": "item-rating"})["title"]

            e_item_number_rating = item_info.find("a", {"class": "item-rating"}).span.text.replace("(", "").replace(")", "")

            e_item_description = item_info.find("a", {"class": "item-title"}).text

            item_features = item_info.findAll("ul", {"class": "item-features"})[0]
            e_item_features = []
            for litag in item_features.find_all('li'):
                e_item_features.append(litag.text)
            e_item_features = str(e_item_features)
            item_action = container.find("div", {"class": "item-action"})

            e_item_price = item_action.find("li", {"class": "price-current"}).text.split('(')[0].strip()

            try:
                e_item_offers_available = item_action.find("li", {"class": "price-current"}).text.split('(')[1].replace(')', '')
            except IndexError:
                e_item_offers_available = 'null'


            e_item_price_ship = item_action.find("li", {"class": "price-ship"}).text

            e_item_stock_id = container.find("div", {"class": "item-stock"})["id"]

            count += 1
            rows = str(count) + ";" + e_item_stock_id + ";" + e_image_url + ";" + e_item_brand + ";" + e_item_rating +";" + e_item_number_rating + ";" + e_item_description + ";" + e_item_features + ";" + e_item_price + ";" + e_item_offers_available + ";" + e_item_price_ship + "\n"

            e_item_stock_id = e_image_url = e_item_brand = e_item_number_rating = e_item_description = e_item_features = e_item_price = e_item_offers_available = e_item_price_ship = ''

            #print(rows) for testing
            
            # write rows in file
            f.write(rows)

        except:
            continue

f.close()


